# Microring filter algorithm for tuning/stabilisation

We define a MRF class which will produce microring filters objects. These objects behave just like microring filters, and it is possible to apply bias values to tune them. 

In [185]:
import numpy as np
from scipy.optimize import minimize

# Microring filter object
class MRF(object):
    """Microring filter object"""
    
    # amplitude of the phase misadjustment.
    var_phase = np.pi/8
    
    def __init__(self, number_rings):
        self.number_rings = number_rings
        self.applied_bias = [0.] * number_rings
        self.phase = np.squeeze((-np.pi/8 + (np.pi/8-(-np.pi/8)) * np.random.random_sample((1, 5))).tolist())
        
    def apply_bias(self,ring_number,bias_value):
        """Set the bias for the ring #[ring_number] at [bias_value]"""
        self.applied_bias[ring_number-1] = bias_value
        
    def measure_power(self):
        """Return a scalar which is a function of applied_bias and phase"""
        return 1-np.linalg.norm(self.applied_bias-self.phase)
    
    def test_MRF(self,bias_list):
        """Return the measured power for a given set of bias list"""
        for i in range(0,self.number_rings):
            self.apply_bias(i+1,bias_list[i])
        return self.measure_power()
    
    def minimize_MRF(self,bias_list):
        """Minimization optimisation function for the MRF object"""
        return -1 * self.test_MRF(bias_list)

This section of code contains the algorithm to tune the microring filter.

In [186]:
# Coordinates descent Algorithm 
def CoordsDescent(MRF,number_iter):
    """Coarse tuning of a MRF object using the coordinates descent algorithm."""
    
    # Possible bias values
    bias_min = -np.pi/8
    bias_max = np.pi/8
    bias_points = 100
    bias_testpoints = np.linspace(bias_min,bias_max,bias_points).tolist()
    
    for i in range(1,number_iter+1): # For each iteration
        for j in range(0,MRF.number_rings): # For each ring
            power_list = []
            for k in bias_testpoints: # For each bias value
                MRF.apply_bias(j,k)
                power_list.append(MRF.measure_power())
            MRF.apply_bias(j,bias_testpoints[power_list.index(max(power_list))])

# Nelder Mead simplex algorithm
def NelderMead(MRF):
    """Fine tuning of a MRF object using the Nelder Mead simplex algorithm"""
    
    x0 = MRF.applied_bias
    res = minimize(MRF.minimize_MRF, x0, method='Nelder-Mead', tol=1e-6, options={'disp': True})
    return res.x

def tuneMRF(MRF):
    """Tune/stabilise the MRF object using coarse + fine algorithms"""
    
    CoordsDescent(MRF,2) # Coordinates descent
    NelderMead(MRF) # Nelder Mead

This last section is an example to show how the code works and how it reaches the value.

In [187]:
myfil = MRF(5)
print myfil.phase
print myfil.applied_bias
print myfil.measure_power()
tuneMRF(myfil)
print myfil.phase
print myfil.applied_bias
print myfil.measure_power()

[ 0.03620423  0.02940442  0.33332936 -0.08134395  0.14772456]
[0.0, 0.0, 0.0, 0.0, 0.0]
0.623538567999
Optimization terminated successfully.
         Current function value: -0.999999
         Iterations: 118
         Function evaluations: 200
[ 0.03620423  0.02940442  0.33332936 -0.08134395  0.14772456]
[0.036204553941050974, 0.029404462799956059, 0.33332950827003949, -0.081344319237629914, 0.14772425124680647]
0.999999398638
